In [1]:
import pandas as pd
import requests
import numpy as np
import json
import re
from gvol import GVol
from plotly.subplots import make_subplots
from datetime import datetime
import plotly.graph_objects as go

GVOL_API_KEY = "GVOL_API_KEY"
gvol_client = GVol(header='gvol-lite',gvol_api_key=GVOL_API_KEY)
url = "https://app.pinkswantrading.com/graphql"
pd.set_option('display.max_columns', None)

In [2]:
# Prepare the payload for the data request
# This is a GraphQL query that fetches live fixed delta surface data from the Deribit exchange for BTC
payload="{\"query\":\"query ZScoreDvol($symbol: SymbolEnumType, $exchange: ExchangeEnumType, $dateStart: String, $dateEnd: String) {ZScoreDvol(symbol: $symbol, exchange: $exchange dateStart: $dateStart dateEnd: $dateEnd) { date currency ln clse zScore dvol }   }\",\"variables\":{\"symbol\":\"BTC\",\"exchange\":\"deribit\",\"dateStart\":\"2020-01-01\",\"dateEnd\":\"2023-07-01\"}}"

# Set up the request headers
# The x-oracle header is used for authentication
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

# Send the GET request to the specified URL
# The response will contain the requested data
response = requests.request("GET", url, headers=headers, data=payload)

# Parse the response text into JSON format
data = json.loads(response.text)

# Convert the JSON data into a pandas DataFrame
# The json_normalize function is used to flatten the JSON into a table
z_scores = pd.json_normalize(data['data']['ZScoreDvol'])

# Convert 'date' column to datetime and set as index
z_scores['date'] = pd.to_datetime(z_scores['date'], unit='ms')
z_scores.set_index('date', inplace=True)
z_scores.head()

,currency,ln,clse,zScore,dvol
date,,,,,
2023-06-30,BTC,0.080,30470.45,0.03,43.52
2023-06-29,BTC,1.221,30446.61,0.48,48.44
2023-06-28,BTC,-2.032,30080.94,-0.79,48.92
2023-06-27,BTC,1.382,30692.14,0.53,49.51
2023-06-26,BTC,-0.684,30269.26,-0.26,49.51


In [3]:
def plot_close(df, lower_percentile, upper_percentile):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=z_scores.index,
        y=z_scores['dvol'],
        mode='lines',
        name='Closing Price',
        line=dict(color='white')
    ))

    # Calculate the percentile values
    upper_value = z_scores['dvol'].quantile(upper_percentile)
    lower_value = z_scores['dvol'].quantile(lower_percentile)

    # Add shaded region for values above upper percentile
    fig.add_shape(
        type='rect', 
        xref='x', yref='y', 
        x0=df.index.min(), y0=upper_value, 
        x1=df.index.max(), y1=z_scores['dvol'].max(), 
        fillcolor='Red', opacity=0.2, 
        line_width=0,
    )

    # Add shaded region for values below lower percentile
    fig.add_shape(
        type='rect', 
        xref='x', yref='y', 
        x0=df.index.min(), y0=lower_value, 
        x1=df.index.max(), y1=z_scores['dvol'].min(), 
        fillcolor='Green', opacity=0.2, 
        line_width=0,
    )

    # Add annotations for percentiles
    fig.add_annotation(
        x=.99, y=0.98, xref='paper', yref='paper',
        text=f'Upper {upper_percentile*100} percentile',
        showarrow=False, font=dict(color='Red')
    )
    fig.add_annotation(
        x=0.99, y=0.925, xref='paper', yref='paper',
        text=f'Lower {lower_percentile*100} percentile',
        showarrow=False, font=dict(color='Green')
    )

    fig.update_layout(
        title="Bitcoin DVOL Closing Prices: High and Low Percentiles",
        yaxis_title="Closing Price ($USD)",
        template='plotly_dark',
        yaxis=dict(range=[z_scores['dvol'].min(), z_scores['dvol'].max()]),  # Set y-axis range
    )

    fig.show()

In [4]:
plot_close(z_scores, 0.05, 0.95)

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

AD_COLORS = ['#231342', '#882c91', '#e00d8e', '#4c61ab', '#87ddf3']
AD_COLORS_FB = ['#4c61ab', '#e00d8e', '#89bed8', '#882c91', '#f16623']
AD_COLORS_3 = ['#1c3664', '#89bed8', '#f16623']


def plot_close(z_scores, lower_percentile, upper_percentile):
    # Create subplots: use two rows and one column
    fig = make_subplots(rows=2, cols=1)

    # Line plot of z_scores['dvol']
    fig.add_trace(go.Scatter(
        x=z_scores.index,
        y=z_scores['dvol'],
        mode='lines',
        name='DVOL',
        line=dict(color='black')  # Set line color to black
    ), row=1, col=1)

    # Calculate the percentile values
    upper_value = z_scores['dvol'].quantile(upper_percentile)
    lower_value = z_scores['dvol'].quantile(lower_percentile)

    # Calculate the y-axis range for the line plot
    yaxis_range = [z_scores['dvol'].min() - 0.20 * z_scores['dvol'].min(), z_scores['dvol'].max()]

    # Add shaded region for values above upper percentile
    fig.add_shape(
        type='rect',
        xref='x1', yref='y1',
        x0=z_scores.index.min(), y0=upper_value,
        x1=z_scores.index.max(), y1=yaxis_range[1],
        fillcolor=AD_COLORS_FB[1], opacity=0.2,  # Use Amberdata color for fill color and set opacity to 0.2
        line_width=0,
    )

    # Add shaded region for values below lower percentile
    fig.add_shape(
        type='rect',
        xref='x1', yref='y1',
        x0=z_scores.index.min(), y0=yaxis_range[0],
        x1=z_scores.index.max(), y1=lower_value,
        fillcolor=AD_COLORS_FB[4], opacity=0.2,  # Use Amberdata color for fill color and set opacity to 0.2
        line_width=0,
    )

    # Bar plot of z_scores['zScore']
    fig.add_trace(go.Bar(
        x=z_scores.index,
        y=z_scores['zScore'],
        name='Z-Score',
        marker_color='black',  # Set marker color to black
        showlegend=False
    ), row=2, col=1)

    # Calculate the y-axis range for the bar chart
    yaxis2_range = [z_scores['zScore'].min() - 0.20 * z_scores['zScore'].min(), z_scores['zScore'].max()]

    # Add annotations for percentiles
    fig.add_annotation(
        x=0.99, y=0.98, xref='paper', yref='paper',
        text=f'Upper {upper_percentile * 100} percentile',
        showarrow=False, font=dict(color='black')
    )
    fig.add_annotation(
        x=0.99, y=0.95, xref='paper', yref='paper',
        text=f'Lower {lower_percentile * 100} percentile',
        showarrow=False, font=dict(color='black')
    )

    # Update layout
    fig.update_layout(
        title="Bitcoin DVOL and Z-Scores: High and Low Percentiles",
        yaxis_title="DVOL",
        yaxis2_title="Z-Score",
        template='plotly_white',  # Use the 'plotly_white' template for a white background
        yaxis=dict(range=yaxis_range),
        yaxis2=dict(range=yaxis2_range),
        height=700,
        showlegend=False
    )

    fig.show()

In [6]:
plot_close(z_scores, 0.05, 0.95)


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

AD_COLORS = ['#231342', '#882c91', '#e00d8e', '#4c61ab', '#87ddf3']
AD_COLORS_FB = ['#4c61ab', '#e00d8e', '#89bed8', '#882c91', '#f16623']
AD_COLORS_3 = ['#1c3664', '#89bed8', '#f16623']


def plot_close(z_scores, lower_percentile, upper_percentile):
    # Create subplots: use two rows and one column
    fig = make_subplots(rows=2, cols=1)

    # Line plot of z_scores['dvol']
    fig.add_trace(go.Scatter(
        x=z_scores.index,
        y=z_scores['dvol'],
        mode='lines',
        name='DVOL',
        line=dict(color='black')  # Set line color to black
    ), row=1, col=1)

    # Calculate the percentile values
    upper_value = z_scores['dvol'].quantile(upper_percentile)
    lower_value = z_scores['dvol'].quantile(lower_percentile)

    # Calculate the y-axis range for the line plot
    yaxis_range = [z_scores['dvol'].min() - 0.20 * z_scores['dvol'].min(), z_scores['dvol'].max()]

    # Add shaded region for values above upper percentile
    fig.add_shape(
        type='rect',
        xref='x1', yref='y1',
        x0=z_scores.index.min(), y0=upper_value,
        x1=z_scores.index.max(), y1=yaxis_range[1],
        fillcolor=AD_COLORS_3[2], opacity=0.2,  # Use Amberdata color for fill color and set opacity to 0.2
        line_width=0,
    )

    # Add shaded region for values below lower percentile
    fig.add_shape(
        type='rect',
        xref='x1', yref='y1',
        x0=z_scores.index.min(), y0=yaxis_range[0],
        x1=z_scores.index.max(), y1=lower_value,
        fillcolor=AD_COLORS_3[1], opacity=0.2,  # Use Amberdata color for fill color and set opacity to 0.2
        line_width=0,
    )

    # Bar plot of z_scores['zScore']
    fig.add_trace(go.Bar(
        x=z_scores.index,
        y=z_scores['zScore'],
        name='Z-Score',
        marker_color='black',  # Set marker color to black
        showlegend=False
    ), row=2, col=1)

    # Calculate the y-axis range for the bar chart
    yaxis2_range = [z_scores['zScore'].min() - 0.20 * z_scores['zScore'].min(), z_scores['zScore'].max()]

    # Add annotations for percentiles
    fig.add_annotation(
        x=0.99, y=0.99, xref='paper', yref='paper',
        text=f'Upper {upper_percentile * 100} percentile',
        showarrow=False, font=dict(color='black')
    )
    fig.add_annotation(
        x=0.99, y=0.95, xref='paper', yref='paper',
        text=f'Lower {lower_percentile * 100} percentile',
        showarrow=False, font=dict(color='black')
    )

    fig.update_layout(
        title="Bitcoin DVOL and Z-Scores: High and Low Percentiles",
        yaxis_title="DVOL",
        yaxis2_title="Z-Score",
        template='plotly_white',  # Use the 'plotly_white' template for a white background
        yaxis=dict(range=yaxis_range),
        yaxis2=dict(range=yaxis2_range),
        height=700,
        showlegend=False
    )

    fig.show()
    fig.write_image('dvol_zscore_percentiles.svg', engine='kaleido', width=900, height=500)

In [10]:
plot_close(z_scores, 0.05, 0.95)